In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import seaborn as sns
import itertools
from itertools import groupby
import os
import re
from pathlib import Path
import ast

In [2]:
from fractions import Fraction
# Convert the string to a Fraction object
def convert_to_float(value):
    value_f = Fraction(value)
    return float(value_f)

In [ ]:
pathlist = Path('../../asc_data').rglob('*.asc')
task_data_df = []
df_final = pd.DataFrame()
time_delay_non_cued = []
placement_position_saccade = []
# for all participant
for path in pathlist:
    participant= re.search(r"(P\d+)", os.path.basename(str(path.stem))).group(1)
    f = open(str(path), 'r') # 'r' = read
    print(participant)
    lines = f.read()
    f.close()
    # j = 4
    # n = 30
    indices = []
    current_start_index = 0
    not_list = []
    for i in range(10):
        
        time_delay_saccade = re.findall(r'time delay stimulus=(.*)',lines[0:-1])
        time_delay_saccade_ = [format(convert_to_float(data), ".2f") for data in time_delay_saccade][0:10:1]

        placement_position_saccade = re.findall(r'placement position=(.*)',lines[0:-1])[0:10:1]
        placement_position_saccade_ = [ast.literal_eval(data) for data in placement_position_saccade]

        index_init1 = re.search(r'saccade_task_start',lines[current_start_index:-1]).span()[1]
        index_init2 = re.search(r'\n',lines[current_start_index + index_init1:-1]).span()[1]
        index_init =  current_start_index + index_init1 + index_init2

        index_finit1 = re.search(r'saccade_task_end',lines[index_init:-1]).span()[0]
        index_finit = index_init + index_finit1

        indices.append([index_init, index_finit-1])
        current_start_index = index_finit

        # if(i + 1) % 5 == 0:
        #     j = int(j/2)

        new_vec = lines[index_init: index_finit-1].split("\n")
        new_new_vec = []
        #0 = blink, 1 fixation, 2 saccade
        stateL = 0
        stateR = 0
        for k in range(1,len(new_vec)):
            la = new_vec[k].split("\t")
            if len(la) == 1:
                if 'SSACC L' in la[0]:
                    stateL = 2
                if 'SFIX L' in la[0]:
                    stateL = 1
                if 'SBLINK L' in la[0]:
                    stateL = 0
                if 'SSACC R' in la[0]:
                    stateR = 2
                if 'SFIX R' in la[0]:
                    stateR = 1
                if 'SBLINK R' in la[0]:
                    stateR = 0
            if len(la) == 9:
                if any(marker in la[0] for marker in ['ESACC L', 'EFIX L', 'EBLINK L', 'ESACC R', 'EFIX R', 'EBLINK R']):
                    continue
                else:
                    la[-1] = stateL
                    la.append(stateR)
                    new_new_vec.append(la)
            if len(la) == 6:
                if any(marker in la[0] for marker in ['ESACC L', 'EFIX L', 'EBLINK L', 'ESACC R', 'EFIX R', 'EBLINK R']):
                    continue
                else:
                    la[-1] = stateL
                    la.append(stateR)
                    new_new_vec.append(la)

        ls = [[np.nan if x == '   .' else x for x in y] for y in new_new_vec]
        if len(ls)>0:
            ls = np.swapaxes(ls,0,1)
            current_trial = np.asarray(ls, dtype=float)
            # if participant =='P205':
            # print(len(current_trial))
            if len(current_trial) == 10:
                my_df = pd.DataFrame(list(np.swapaxes(np.array([current_trial[0][:],current_trial[4][:],current_trial[5][:],current_trial[-1][:]]),0,1)), columns=['Time','RX','RY','state'])
            else:
                # print(current_trial[7][:])
                my_df = pd.DataFrame(list(np.swapaxes(np.array([current_trial[0][:],current_trial[1][:],current_trial[2][:],current_trial[5][:]]),0,1)), columns=['Time','RX','RY','state'])
            my_df['participants'] = participant
            my_df['time_delay'] = time_delay_saccade_[i]
            my_df['X'] = placement_position_saccade_[i][0]
            my_df['Y'] = placement_position_saccade_[i][1]
            my_df['interval_num'] = i+1
            # my_df['FREQ']=j

            task_data_df.append(my_df)
            df_final = pd.concat([df_final,my_df], ignore_index=True)
        else:
            not_list.append(i)

df_final.to_csv("../../processed_data/processed_data_asc_task_sacccades_all.csv", index=False)
    

P004
P006
P008
P020
P036
P063
P065
P070
P072
P074
P075
P076
P085
P086
P093
P094
P103
P106
P109
P112
P150
P157
P159
P160
P170
P171
P172
P173
P176
P177
P179
P182
P183
P184
P186
P188
P190
P194
P199
P201
P202
P203
P204
P205
P206
P207
P208
P210


In [6]:
df_final

,Time,RX,RY,state,participants,time_delay,X,Y,interval_num
0,84222713.0,1127.2,766.9,0.0,P004,0.83,0.65,0,1
1,84222714.0,1128.1,766.5,0.0,P004,0.83,0.65,0,1
2,84222715.0,1128.8,765.7,0.0,P004,0.83,0.65,0,1
3,84222716.0,1129.6,765.0,0.0,P004,0.83,0.65,0,1
4,84222717.0,1129.8,763.9,0.0,P004,0.83,0.65,0,1
...,...,...,...,...,...,...,...,...,...
1035442,428141442.0,1609.0,545.3,1.0,P210,1.51,0.65,0,10
1035443,428141443.0,1608.4,545.2,1.0,P210,1.51,0.65,0,10
1035444,428141444.0,1608.0,548.3,1.0,P210,1.51,0.65,0,10
1035445,428141445.0,1607.5,551.1,1.0,P210,1.51,0.65,0,10
